# Predicting sentiment from product reviews


The goal of this first notebook is to explore logistic regression and feature engineering with existing GraphLab functions.

In this notebook you will use product review data from Amazon.com to predict whether the sentiments about a product (from its reviews) are positive or negative.

* Train a logistic regression model to predict the sentiment of product reviews.
* Inspect the weights (coefficients) of a trained logistic regression model.
* Make a prediction (both class and probability) of sentiment for a new product review.
* Given the logistic regression weights, predictors and ground truth labels, write a function to compute the **accuracy** of the model.
* Inspect the coefficients of the logistic regression model and interpret their meanings.
* Compare multiple logistic regression models.

Let's get started!
    

In [1]:
from __future__ import division
import math
import string
import numpy as np
import pandas as pd

In [2]:
dtype_dict = {'name':str,
              'review':str, 
              'rating':int}

In [3]:
products = pd.read_csv('amazon_baby.csv', dtype = dtype_dict)

## Build the word count vector for each review

Let us explore a specific example of a baby product.


In [4]:
products_matrix = products.values
print(products_matrix.shape)
print(products_matrix[269])

(183531, 3)
['The First Years Massaging Action Teether' 'A favorite in our house!' 5]


Now, we will perform 2 simple data transformations:

1. Remove punctuation using [Python's built-in](https://docs.python.org/2/library/string.html) string functionality.
2. Transform the reviews into word-counts.

**Aside**. In this notebook, we remove all punctuations for the sake of simplicity. A smarter approach to punctuations would preserve phrases such as "I'd", "would've", "hadn't" and so forth. See [this page](https://www.cis.upenn.edu/~treebank/tokenization.html) for an example of smart handling of punctuations.

In [5]:
products = products.fillna({'review':''})  # fill in N/A's in the review column

In [6]:
products['review_without_punctuation']=products['review'].str.replace('[{}]'.format(string.punctuation), '')

In [7]:
print(products['review'][0])

These flannel wipes are OK, but in my opinion not worth keeping.  I also ordered someImse Vimse Cloth Wipes-Ocean Blue-12 countwhich are larger, had a nicer, softer texture and just seemed higher quality.  I use cloth wipes for hands and faces and have been usingThirsties 6 Pack Fab Wipes, Boyfor about 8 months now and need to replace them because they are starting to get rough and have had stink issues for a while that stripping no longer handles.


In [8]:
print(products['review_without_punctuation'][0])

These flannel wipes are OK but in my opinion not worth keeping  I also ordered someImse Vimse Cloth WipesOcean Blue12 countwhich are larger had a nicer softer texture and just seemed higher quality  I use cloth wipes for hands and faces and have been usingThirsties 6 Pack Fab Wipes Boyfor about 8 months now and need to replace them because they are starting to get rough and have had stink issues for a while that stripping no longer handles


In [9]:
print(products.head(0))

Empty DataFrame
Columns: [name, review, rating, review_without_punctuation]
Index: []


## Extract sentiments

We will **ignore** all reviews with *rating = 3*, since they tend to have a neutral sentiment.

In [10]:
products = products[products['rating'] != 3]

In [11]:
products['sentiments'] = products['rating'].apply(lambda rating: 1 if rating > 3 
                                                 else -1)

In [12]:
import json

In [13]:
print(products.shape)

(166752, 5)


In [14]:
list(products.columns.values)

['name', 'review', 'rating', 'review_without_punctuation', 'sentiments']

In [15]:
with open('test-idx.json', 'r') as f:
    test_data = json.load(f)
print(len(test_data))
test_data = pd.DataFrame(products.values[test_data])
test_data.columns = list(products.columns.values)

print(test_data.shape)

33336
(33336, 5)


In [16]:
with open('train-idx.json', 'r') as f:
    train_data= json.load(f)
print(len(train_data))

train_data = pd.DataFrame(products.values[train_data])
train_data.columns = list(products.columns.values)

print(train_data.shape)

133416
(133416, 5)


## Build the word count vector for each review

We will now compute the word count for each word that appears in the reviews. A vector consisting of word counts is often referred to as bag-of-word features. Since most words occur in only a few reviews, word count vectors are sparse. For this reason, scikit-learn and many other tools use sparse matrices to store a collection of word count vectors. Refer to appropriate manuals to produce sparse word count vectors. General steps for extracting word count vectors are as follows:

* Learn a vocabulary (set of all words) from the training data. Only the words that show up in the training data will be considered for feature extraction.

* Compute the occurrences of the words in each review and collect them into a row vector.

* Build a sparse matrix where each row is the word count vector for the corresponding review. Call this matrix train_matrix.

* Using the same mapping between words and columns, convert the test data into a sparse matrix test_matrix.

The following cell uses CountVectorizer in scikit-learn. Notice the token_pattern argument in the constructor.

In [17]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')
# Use this token pattern to keep single-letter words
# First, learn vocabulary from the training data and assign columns to words
# Then convert the training data into a sparse matrix
train_matrix = vectorizer.fit_transform(train_data['review_without_punctuation'])
# Second, convert the test data into a sparse matrix, using the same word-column mapping
test_matrix = vectorizer.transform(test_data['review_without_punctuation'])

In [18]:
print(train_matrix.shape)

(133416, 121706)


## Train a sentiment classifier with logistic regression

We will now use logistic regression to create a sentiment classifier on the training data.

Learn a logistic regression classifier using the training data. If you are using scikit-learn, you should create an instance of the LogisticRegression class and then call the method ```fit()``` to train the classifier. This model should use the sparse word count matrix (```train_matrix```) as features and the column sentiment of train_data as the target. Use the default values for other parameters. Call this model sentiment_model.

In [19]:
from sklearn import linear_model

In [23]:
train_output = train_data['sentiments']
train_output=train_output.astype('int')
type(train_output)

pandas.core.series.Series

In [52]:
print(train_matrix.shape)
print(train_data.shape)

(133416, 121706)
(133416, 5)


In [24]:
sentiment_model = linear_model.LogisticRegression()
sentiment_model.fit(train_matrix, train_output)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

There should be over 100,000 coefficients in this sentiment_model. Recall from the lecture that positive weights ```w_j``` correspond to weights that cause positive sentiment, while negative weights correspond to negative sentiment. Calculate the number of positive (```>= 0```, which is actually nonnegative) coefficients.

**Quiz question: How many weights are >= 0?**

In [27]:
print((sentiment_model.coef_>0).sum())
print((sentiment_model.coef_<0).sum())
print((sentiment_model.coef_==0).sum())

85837
35869
0


## Making predictions with logistic regression

Now that a model is trained, we can make predictions on the test data. In this section, we will explore this in the context of 3 data points in the test data. Take the 11th, 12th, and 13th data points in the test data and save them to sample_test_data. The following cell extracts the three data points from the SFrame test_data and print their content:

In [32]:
sample_test_data = test_data[10:13]
print(sample_test_data.shape)

sample_test_matrix = test_matrix[10:13]

(3, 5)


In [30]:
print(sample_test_data)

                                                 name  \
10                          Our Baby Girl Memory Book   
11  Wall Decor Removable Decal Sticker - Colorful ...   
12  New Style Trailing Cherry Blossom Tree Decal R...   

                                               review rating  \
10  Absolutely love it and all of the Scripture in...      5   
11  Would not purchase again or recommend. The dec...      2   
12  Was so excited to get this product for my baby...      1   

                           review_without_punctuation sentiments  
10  Absolutely love it and all of the Scripture in...          1  
11  Would not purchase again or recommend The deca...         -1  
12  Was so excited to get this product for my baby...         -1  


In [34]:
sample_test_output= sample_test_data['sentiments']
sample_test_output=sample_test_output.astype('int')

sample_predictions = sentiment_model.predict(sample_test_matrix)

In [35]:
print(sample_predictions)

[ 1 -1 -1]


We will now make a **class** prediction for the **sample_test_data**. The `sentiment_model` should predict **+1** if the sentiment is positive and **-1** if the sentiment is negative. Recall from the lecture that the **score** (sometimes called **margin**) for the logistic regression model  is defined as:

$$
\mbox{score}_i = \mathbf{w}^T h(\mathbf{x}_i)
$$ 

where $h(\mathbf{x}_i)$ represents the features for example $i$.  We will write some code to obtain the **scores** using GraphLab Create. For each row, the **score** (or margin) is a number in the range **[-inf, inf]**.

In [37]:
scores = sentiment_model.decision_function(sample_test_matrix)
print(scores)

[  5.59355542  -3.13902441 -10.40249874]


### Predicting sentiment

These scores can be used to make class predictions as follows:

$$
\hat{y} = 
\left\{
\begin{array}{ll}
      +1 & \mathbf{w}^T h(\mathbf{x}_i) > 0 \\
      -1 & \mathbf{w}^T h(\mathbf{x}_i) \leq 0 \\
\end{array} 
\right.
$$

Using scores, write code to calculate $\hat{y}$, the class predictions:

**Checkpoint**: Make sure your class predictions match with the one obtained from GraphLab Create.

### Probability predictions

Recall from the lectures that we can also calculate the probability predictions from the scores using:
$$
P(y_i = +1 | \mathbf{x}_i,\mathbf{w}) = \frac{1}{1 + \exp(-\mathbf{w}^T h(\mathbf{x}_i))}.
$$

Using the variable **scores** calculated previously, write code to calculate the probability that a sentiment is positive using the above formula. For each row, the probabilities should be a number in the range **[0, 1]**.

In [40]:
p = 1/(1+np.exp(-scores))
print(p)
print(p>=0.5)

[  9.96292028e-01   4.15259321e-02   3.03556134e-05]
[ True False False]


# Find the most positive (and negative) review

We now turn to examining the full test dataset, **test_data**, to form predictions on all of the test data points for faster performance.

Using the `sentiment_model`, find the 20 reviews in the entire **test_data** with the **highest probability** of being classified as a **positive review**. We refer to these as the "most positive reviews."

To calculate these top-20 reviews, use the following steps:
1.  Make probability predictions on **test_data** using the `sentiment_model`. (**Hint:** When you call `.predict` to make predictions on the test data, use option `output_type='probability'` to output the probability rather than just the most likely class.)
2.  Sort the data according to those predictions and pick the top 20. (**Hint:** You can use the `.topk` method on an SFrame to find the top k rows sorted according to the value of a specified column.)

In [41]:
scores_all = sentiment_model.decision_function(test_matrix)


In [45]:
sorted_scores_all_index = np.argsort(-scores_all)

In [48]:
print(scores_all[sorted_scores_all_index[0:20]])
print(sorted_scores_all_index[0:20])

[ 53.55701871  51.96795627  48.23815669  46.3150606   44.12976888
  43.08579024  41.93927841  40.82291962  40.55038703  40.42484511
  40.1667546   39.86017685  38.87524658  38.11059032  37.59445731
  37.55648131  36.24387372  36.19483965  35.35932833  35.22911277]
[18112 15732 24286 25554 24899  9125 21531 32782  9555 30535 14482 30634
 17558 26830 20743 11923  4140 30076 26838 33060]


In [51]:
top_20_test_name = test_data['name'][sorted_scores_all_index[0:20]]

In [52]:
print(top_20_test_name)

18112    Infantino Wrap and Tie Baby Carrier, Black Blu...
15732      Baby Einstein Around The World Discovery Center
24286                    Britax 2012 B-Agile Stroller, Red
25554           Diono RadianRXT Convertible Car Seat, Plum
24899           Graco Pack 'n Play Element Playard - Flint
9125            P'Kolino Silly Soft Seating in Tias, Green
21531    Roan Rocco Classic Pram Stroller 2-in-1 with B...
32782        Mamas &amp; Papas 2014 Urbo2 Stroller - Black
9555     Evenflo X Sport Plus Convenience Stroller - Ch...
30535    Buttons Cloth Diaper Cover - One Size - 8 Colo...
14482    Simple Wishes Hands-Free Breastpump Bra, Pink,...
30634    Graco FastAction Fold Jogger Click Connect Str...
17558    Freemie Hands-Free Concealable Breast Pump Col...
26830    Baby Jogger City Mini GT Single Stroller, Shad...
20743    Fisher-Price Cradle 'N Swing,  My Little Snuga...
11923         Evenflo 6 Pack Classic Glass Bottle, 4-Ounce
4140        Britax Decathlon Convertible Car Seat, Tiffa

In [54]:
low_20_test_name = test_data['name'][sorted_scores_all_index[-20:]]
print(low_20_test_name)

5831                Regalo My Cot Portable Bed, Royal Blue
15062        Thirsties Hemp Inserts 2 Pack, Small 6-18 Lbs
205                  Safety 1st Deluxe 4-in-1 Bath Station
28120    VTech Communications Safe &amp; Sound Digital ...
7310     Chicco Cortina KeyFit 30 Travel System in Adve...
27231                     NUK Cook-n-Blend Baby Food Maker
31226    Belkin WeMo Wi-Fi Baby Monitor for Apple iPhon...
13751           Peg-Perego Tatamia High Chair, White Latte
10814               Ellaroo Mei Tai Baby Carrier - Hershey
1810          Cosco Alpha Omega Elite Convertible Car Seat
1942                     Philips AVENT Newborn Starter Set
20594    Motorola Digital Video Baby Monitor with Room ...
14711                Cloth Diaper Sprayer--styles may vary
9655                   Safety 1st High-Def Digital Monitor
17069    The First Years True Choice P400 Premium Digit...
28184    VTech Communications Safe &amp; Sounds Full Co...
8818     Adiri BPA Free Natural Nurser Ultimate Bottle .

**Quiz Question**: Which of the following products are represented in the 20 most negative reviews?  [multiple choice]

## Compute accuracy of the classifier

We will now evaluate the accuracy of the trained classifier. Recall that the accuracy is given by


$$
\mbox{accuracy} = \frac{\mbox{# correctly classified examples}}{\mbox{# total examples}}
$$

This can be computed as follows:

* **Step 1:** Use the trained model to compute class predictions (**Hint:** Use the `predict` method)
* **Step 2:** Count the number of data points when the predicted class labels match the ground truth labels (called `true_labels` below).
* **Step 3:** Divide the total number of correct predictions by the total number of data points in the dataset.

Complete the function below to compute the classification accuracy:

In [55]:
def get_classification_accuracy(model, data, true_labels):
    # First get the predictions
    
    predictions = model.predict(data)
    
    # Compute the number of correctly classified examples
    accuracy = (predictions == true_labels).sum()/(len(true_labels))

    # Then compute accuracy by dividing num_correct by total number of examples
    
    return accuracy

In [56]:
test_output = test_data['sentiments']
test_output = test_output.astype('int')

In [57]:
get_classification_accuracy(sentiment_model, test_matrix, test_output)

0.93238540916726664

# Learn another classifier with fewer words

There were a lot of words in the model we trained above. We will now train a simpler logistic regression model using only a subset of words that occur in the reviews. For this assignment, we selected a 20 words to work with. These are:

In [58]:
significant_words = ['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 
      'well', 'able', 'car', 'broke', 'less', 'even', 'waste', 'disappointed', 
      'work', 'product', 'money', 'would', 'return']

Compute a new set of word count vectors using only these words. The ```CountVectorizer``` class has a parameter that lets you limit the choice of words when building word count vectors:

In [59]:
vectorizer_word_subset = CountVectorizer(vocabulary=significant_words) # limit to 20 words
train_matrix_word_subset = vectorizer_word_subset.fit_transform(train_data['review_without_punctuation'])
test_matrix_word_subset = vectorizer_word_subset.transform(test_data['review_without_punctuation'])

## Train a logistic regression model on a subset of data
We will now build a classifier with **word_count_subset** as the feature and **sentiment** as the target. 

In [60]:
simple_model = linear_model.LogisticRegression()
simple_model.fit(train_matrix_word_subset, train_output)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

We can compute the classification accuracy using the `get_classification_accuracy` function you implemented earlier.

In [61]:
get_classification_accuracy(simple_model, test_matrix_word_subset, test_output)

0.86936045116390692

Now, we will inspect the weights (coefficients) of the simple_model:

In [62]:
simple_model.coef_

array([[ 1.36368827,  0.94399786,  1.19253652,  0.08550919,  0.52018333,
         1.50981028,  1.67311805,  0.50375862,  0.19090598,  0.05885356,
        -1.65157706, -0.2095648 , -0.51138168, -2.03369873, -2.34830004,
        -0.62117042, -0.32055792, -0.89803241, -0.36215621, -2.10933278]])

**Quiz Question**: Consider the coefficients of **simple_model**. There should be 21 of them, an intercept term + one for each word in **significant_words**. How many of the 20 coefficients (corresponding to the 20 **significant_words** and *excluding the intercept term*) are positive for the `simple_model`?

In [63]:
print((simple_model.coef_>0).sum())

10


In [74]:
print(np.asarray(simple_model.coef_).shape)


(1, 20)


In [75]:
columns = ['coef']
index = significant_words

simple_model_result = pd.DataFrame(np.transpose(np.asarray(simple_model.coef_)), columns=columns, index=index)

print(simple_model_result)

                  coef
love          1.363688
great         0.943998
easy          1.192537
old           0.085509
little        0.520183
perfect       1.509810
loves         1.673118
well          0.503759
able          0.190906
car           0.058854
broke        -1.651577
less         -0.209565
even         -0.511382
waste        -2.033699
disappointed -2.348300
work         -0.621170
product      -0.320558
money        -0.898032
would        -0.362156
return       -2.109333


**Quiz Question**: Are the positive words in the **simple_model** (let us call them `positive_significant_words`) also positive words in the **sentiment_model**?

In [104]:
print(simple_model_result.index)
list_all_coefficient = np.transpose(sentiment_model.coef_)

Index(['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 'well',
       'able', 'car', 'broke', 'less', 'even', 'waste', 'disappointed', 'work',
       'product', 'money', 'would', 'return'],
      dtype='object')


In [105]:
print(list_all_coefficient)
print(list_all_coefficient.shape)


[[ -1.23330228e+00]
 [  2.37538251e-04]
 [  2.55429984e-02]
 ..., 
 [  1.10903893e-02]
 [  3.24828215e-03]
 [ -7.75234981e-05]]
(121706, 1)


In [106]:
testing_result = 1
counting = 0
for word in simple_model_result.index:
    if simple_model_result.values[counting]>0:
        index = vectorizer.vocabulary_.get(word)
        if list_all_coefficient[index]>0:
            testing_result = testing_result  * 1
        else:
            testing_result = testing_result * 0
    counting +=1
print(testing_result)
        

1


# Comparing models

We will now compare the accuracy of the **sentiment_model** and the **simple_model** using the `get_classification_accuracy` method you implemented above.

First, compute the classification accuracy of the **sentiment_model** on the **train_data**:

In [79]:
sentiment_model = linear_model.LogisticRegression()
sentiment_model.fit(train_matrix, train_output)

0.96746267314265155

In [80]:
get_classification_accuracy(sentiment_model, train_matrix, train_output)

0.96746267314265155

Now, compute the classification accuracy of the **simple_model** on the **train_data**:

In [82]:
get_classification_accuracy(simple_model, train_matrix_word_subset, train_output)

0.8668225700065959

**Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TRAINING set?

Now, we will repeat this exercise on the **test_data**. Start by computing the classification accuracy of the **sentiment_model** on the **test_data**:

Actually, we have done this previously. Here, we just simply call the functions.

In [83]:
get_classification_accuracy(sentiment_model, test_matrix, test_output)

0.93238540916726664

In [84]:
get_classification_accuracy(simple_model, test_matrix_word_subset, test_output)

0.86936045116390692

## Baseline: Majority class prediction

It is quite common to use the **majority class classifier** as the a baseline (or reference) model for comparison with your classifier model. The majority classifier model predicts the majority class for all data points. At the very least, you should healthily beat the majority class classifier, otherwise, the model is (usually) pointless.

What is the majority class in the **train_data**?

In [86]:
num_positive  = (test_output == +1).sum()
num_negative = (test_output == -1).sum()
print(num_positive)
print (num_negative)

28095
5241


Now compute the accuracy of the majority class classifier on **test_data**.

**Quiz Question**: Enter the accuracy of the majority class classifier model on the **test_data**. Round your answer to two decimal places (e.g. 0.76).

In [87]:
num_positive/(num_positive+num_negative)

0.84278257739380846